## Libraries

In [275]:
import pandas as pd
import numpy as np
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report

In [259]:
data_url='https://raw.githubusercontent.com/Kamalesh1512/datasets/main/stock_sentiment.csv'
# reading data
df=pd.read_csv(data_url,encoding='latin')

#converting to datetime
df['Date']=pd.to_datetime(df['Date'])
## set data as index
df.set_index('Date',inplace=True)

In [260]:
df.head()

,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-03,0,A 'hindrance to operations': extracts from the...,Scorecard,Hughes' instant hit buoys Blues,Jack gets his skates on at ice-cold Alex,Chaos as Maracana builds up for United,Depleted Leicester prevail as Elliott spoils E...,Hungry Spurs sense rich pickings,Gunners so wide of an easy target,Derby raise a glass to Strupar's debut double,...,Flintoff injury piles on woe for England,Hunters threaten Jospin with new battle of the...,Kohl's successor drawn into scandal,The difference between men and women,"Sara Denver, nurse turned solicitor",Diana's landmine crusade put Tories in a panic,Yeltsin's resignation caught opposition flat-f...,Russian roulette,Sold out,Recovering a title
2000-01-04,0,Scorecard,The best lake scene,Leader: German sleaze inquiry,"Cheerio, boyo",The main recommendations,Has Cubie killed fees?,Has Cubie killed fees?,Has Cubie killed fees?,Hopkins 'furious' at Foster's lack of Hannibal...,...,On the critical list,The timing of their lives,Dear doctor,Irish court halts IRA man's extradition to Nor...,Burundi peace initiative fades after rebels re...,PE points the way forward to the ECB,Campaigners keep up pressure on Nazi war crime...,Jane Ratcliffe,Yet more things you wouldn't know without the ...,Millennium bug fails to bite
2000-01-05,0,Coventry caught on counter by Flo,United's rivals on the road to Rio,Thatcher issues defence before trial by video,Police help Smith lay down the law at Everton,Tale of Trautmann bears two more retellings,England on the rack,Pakistan retaliate with call for video of Walsh,Cullinan continues his Cape monopoly,McGrath puts India out of their misery,...,South Melbourne (Australia),Necaxa (Mexico),Real Madrid (Spain),Raja Casablanca (Morocco),Corinthians (Brazil),Tony's pet project,Al Nassr (Saudi Arabia),Ideal Holmes show,Pinochet leaves hospital after tests,Useful links
2000-01-06,1,Pilgrim knows how to progress,Thatcher facing ban,McIlroy calls for Irish fighting spirit,Leicester bin stadium blueprint,United braced for Mexican wave,"Auntie back in fashion, even if the dress look...",Shoaib appeal goes to the top,Hussain hurt by 'shambles' but lays blame on e...,England's decade of disasters,...,Putin admits Yeltsin quit to give him a head s...,BBC worst hit as digital TV begins to bite,How much can you pay for...,Christmas glitches,"Upending a table, Chopping a line and Scoring ...","Scientific evidence 'unreliable', defence claims",Fusco wins judicial review in extradition case,Rebels thwart Russian advance,Blair orders shake-up of failing NHS,Lessons of law's hard heart
2000-01-07,1,Hitches and Horlocks,Beckham off but United survive,Breast cancer screening,Alan Parker,Guardian readers: are you all whingers?,Hollywood Beyond,Ashes and diamonds,Whingers - a formidable minority,Alan Parker - part two,...,Most everywhere: UDIs,Most wanted: Chloe lunettes,Return of the cane 'completely off the agenda',From Sleepy Hollow to Greeneland,Blunkett outlines vision for over 11s,"Embattled Dobson attacks 'play now, pay later'...",Doom and the Dome,What is the north-south divide?,Aitken released from jail,Gone aloft


In [261]:
df.shape

(4101, 26)

In [262]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4101 entries, 2000-01-03 to 2016-07-01
Data columns (total 26 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Label   4101 non-null   int64 
 1   Top1    4101 non-null   object
 2   Top2    4101 non-null   object
 3   Top3    4101 non-null   object
 4   Top4    4101 non-null   object
 5   Top5    4101 non-null   object
 6   Top6    4101 non-null   object
 7   Top7    4101 non-null   object
 8   Top8    4101 non-null   object
 9   Top9    4101 non-null   object
 10  Top10   4101 non-null   object
 11  Top11   4101 non-null   object
 12  Top12   4101 non-null   object
 13  Top13   4101 non-null   object
 14  Top14   4101 non-null   object
 15  Top15   4101 non-null   object
 16  Top16   4101 non-null   object
 17  Top17   4101 non-null   object
 18  Top18   4101 non-null   object
 19  Top19   4101 non-null   object
 20  Top20   4101 non-null   object
 21  Top21   4101 non-null   object
 22  Top22 

In [263]:
## checking for missing values
df.isnull().sum()

Label    0
Top1     0
Top2     0
Top3     0
Top4     0
Top5     0
Top6     0
Top7     0
Top8     0
Top9     0
Top10    0
Top11    0
Top12    0
Top13    0
Top14    0
Top15    0
Top16    0
Top17    0
Top18    0
Top19    0
Top20    0
Top21    0
Top22    0
Top23    1
Top24    3
Top25    3
dtype: int64

In [264]:
def handle_missing_data(df):
    if (df.isna().sum().any())>0:
        percent=(df.isnull().sum()/df.shape[0])*100
        if (percent.any()<15) | (percent.any()>75):
            df.dropna(inplace=True)
        else:
            df.fillna(method='bfill',inplace=True)
    print(df.isnull().sum())
handle_missing_data(df)

Label    0
Top1     0
Top2     0
Top3     0
Top4     0
Top5     0
Top6     0
Top7     0
Top8     0
Top9     0
Top10    0
Top11    0
Top12    0
Top13    0
Top14    0
Top15    0
Top16    0
Top17    0
Top18    0
Top19    0
Top20    0
Top21    0
Top22    0
Top23    0
Top24    0
Top25    0
dtype: int64


In [265]:
# checking for duplicate data
def handle_duplicate(data):
    if data.duplicated().sum()>0:
        data.drop_duplicates(inplace=True)
    print(data.duplicated().sum())
handle_duplicate(df)

0


In [266]:
### combine text columns into one
def combine_cols(data,columns):
    data['Text']=data[columns].agg(' '.join,axis=1)
    return data['Text']
df['Text']=combine_cols(df,df.columns[1:])

## Text Pre-processing

In [267]:
data=df[['Label','Text']]
data.head()

,Label,Text
Date,,
2000-01-03,0,A 'hindrance to operations': extracts from the...
2000-01-04,0,Scorecard The best lake scene Leader: German s...
2000-01-05,0,Coventry caught on counter by Flo United's riv...
2000-01-06,1,Pilgrim knows how to progress Thatcher facing ...
2000-01-07,1,Hitches and Horlocks Beckham off but United su...


In [268]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4098 entries, 2000-01-03 to 2016-07-01
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Label   4098 non-null   int64 
 1   Text    4098 non-null   object
dtypes: int64(1), object(1)
memory usage: 96.0+ KB


In [272]:
## checking for imbalance in the dataset
100*(data['Label'].value_counts()/data.shape[0])

Label
1    52.781845
0    47.218155
Name: count, dtype: float64

In [270]:
def preprocess(text):
    # Text Lowercasing
    text = text.lower()

    # Tokenization
    tokens = word_tokenize(text)

    # Noise Removal / Punctuations removal
    tokens = [re.sub(r"[^a-zA-Z0-9]", "", token.strip()) for token in tokens]

    # Stopword Removal
    stop_words = set(stopwords.words("english"))
    tokens = [token for token in tokens if token not in stop_words]

    ## Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return " ".join(tokens)

In [273]:
data['Preprocessed_text']=data['Text'].apply(preprocess)

C:\Users\KAMALESH M\AppData\Local\Temp\ipykernel_36180\2533511564.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Preprocessed_text']=data['Text'].apply(preprocess)


In [274]:
data.head()

,Label,Text,Preprocessed_text
Date,,,
2000-01-03,0,A 'hindrance to operations': extracts from the...,hindrance operation extract leaked report sc...
2000-01-04,0,Scorecard The best lake scene Leader: German s...,scorecard best lake scene leader german sleaz...
2000-01-05,0,Coventry caught on counter by Flo United's riv...,coventry caught counter flo united rival road ...
2000-01-06,1,Pilgrim knows how to progress Thatcher facing ...,pilgrim know progress thatcher facing ban mcil...
2000-01-07,1,Hitches and Horlocks Beckham off but United su...,hitch horlocks beckham united survive breast c...


In [290]:
# Assuming you have a DataFrame df with columns 'text' and 'label'
train_size=int(len(data)*0.75)
train_data=data.iloc[:train_size]
test_data=data.iloc[train_size:]

#training data
X_train=train_data['Preprocessed_text']
y_train=train_data['Label']
# testing data
X_test=test_data['Preprocessed_text']
y_test=test_data['Label']

# Preprocess the text (use the preprocess_text function from the previous code snippet)
X_train_preprocessed = [' '.join(preprocess_text(text)) for text in X_train]
X_test_preprocessed = [' '.join(preprocess_text(text)) for text in X_test]

# Convert text to numerical features using TF-IDF
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_preprocessed)
X_test_tfidf = tfidf_vectorizer.transform(X_test_preprocessed)

# Models
models = {
    'Logistic Regression': LogisticRegression(),
    'Naive Bayes': MultinomialNB(),
    'SVM': SVC(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier()
}

# Train and evaluate models
for model_name, model in models.items():
    model.fit(X_train_tfidf, y_train)

    y_pred_train = model.predict(X_train_tfidf)
    train_accuracy = accuracy_score(y_train, y_pred_train)
    print(f'\n{model_name} Accuracy of training data: {train_accuracy:.2f}')
    
    y_pred_test = model.predict(X_test_tfidf)
    test_accuracy = accuracy_score(y_test, y_pred_test)
    print(f'\n{model_name} Accuracy of testing data: {test_accuracy:.2f}')
    
    # Display classification report
    print(classification_report(y_test, y_pred_test))



Logistic Regression Accuracy of training data: 0.96

Logistic Regression Accuracy of testing data: 0.52
              precision    recall  f1-score   support

           0       0.47      0.27      0.34       476
           1       0.54      0.74      0.62       549

    accuracy                           0.52      1025
   macro avg       0.51      0.50      0.48      1025
weighted avg       0.51      0.52      0.49      1025


Naive Bayes Accuracy of training data: 0.76

Naive Bayes Accuracy of testing data: 0.54
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       476
           1       0.54      1.00      0.70       549

    accuracy                           0.54      1025
   macro avg       0.27      0.50      0.35      1025
weighted avg       0.29      0.54      0.37      1025



C:\Users\KAMALESH M\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\KAMALESH M\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\KAMALESH M\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classificatio


SVM Accuracy of training data: 1.00

SVM Accuracy of testing data: 0.53
              precision    recall  f1-score   support

           0       0.44      0.07      0.13       476
           1       0.53      0.92      0.67       549

    accuracy                           0.53      1025
   macro avg       0.49      0.50      0.40      1025
weighted avg       0.49      0.53      0.42      1025


Random Forest Accuracy of training data: 1.00

Random Forest Accuracy of testing data: 0.51
              precision    recall  f1-score   support

           0       0.46      0.33      0.39       476
           1       0.53      0.66      0.59       549

    accuracy                           0.51      1025
   macro avg       0.50      0.50      0.49      1025
weighted avg       0.50      0.51      0.50      1025


Gradient Boosting Accuracy of training data: 0.83

Gradient Boosting Accuracy of testing data: 0.50
              precision    recall  f1-score   support

           0       0.44 